## Preprocess the Sim2Real dataset

The Sim2Real dataset uses jpeg images to represent wildfire masks. This representation is very lightweight but slow to work with. 
To benchmark faster, we convert them to binary NumPy files. These files are ~100 times heavier and it requires an overhead time to convert from jpeg to them, but once they are created and stored, they allow 50--100x faster operations.

1. Download the dataset from [Sim2Real-Fire GitHub repository](https://github.com/TJU-IDVLab/Sim2Real-Fire).
2. Extract all files from the dataset into your `Dataset` folder
3. Run the preprocessing function to convert from the jpeg representation to the NumPy one, and compute burn maps

In [ ]:
pip install -r requirements.txt
pip install juliacall
pip install gurobipy==12.0.1

In [2]:
import gurobipy as gp
print("Gurobi version:", gp.gurobi.version())

Gurobi version: (12, 0, 1)


In [1]:
# import required modules
import sys
import os
import time
import numpy as np
import time

# Add code to path
module_path = os.path.abspath(".") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)

from dataset import preprocess_sim2real_dataset, load_scenario_npy, compute_and_save_burn_maps_sim2real_dataset
from Strategy import RandomDroneRoutingStrategy, return_no_custom_parameters, SensorPlacementOptimization, RandomSensorPlacementStrategy, DroneRoutingOptimizationExample, MY_DRONE_STRATEGY, MY_SENSOR_STRATEGY, MY_SENSOR_STRATEGY_2
from benchmark import run_benchmark_scenario,run_benchmark_scenarii_sequential, benchmark_on_sim2real_dataset, get_burnmap_parameters
from displays import create_scenario_video

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Initializing the Julia session. This can take up to 1 minute.
initializing the ground sensor julia module
installing packages


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environments/pyjuliapkg/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pyjuliapkg/Project.toml`
  No Changes to `~/.julia/environ

initializing the drone julia module
Julia session initialized.


In [ ]:
# n_max_scenarii_per_layout controls the number of scenarios we convert from jpeg to NumPy files for each layout.
# REQUIRES 400 GB of STOCKAGE and ~15 mins
# preprocess_sim2real_dataset("Dataset/", n_max_scenarii_per_layout=100) 

We can now run the benchmark function on any scenario.
1. We load the scenario using `load_scenario_npy` since it is in preprocessed `npy` format
2. Werun the benchmark using `run_benchmark_scenario` that takes as input:
    - The `scenario`, 
    - The sensor placement strategy and the drone routing strategy
    - A dictionary `custom_initialization_parameters` that contains any custom initialization inputs for your strategy functions (such as the burn map if your strategy needs a burn map as input)
    - A python function `custom_step_parameters_function` that returns a dictionary of custom inputs for your routing function. This function is executed by the benchmarking code at each time step. Your routing function will be called with: `routing(custom_step_parameters_function())` internally

In [2]:
# change values here to change benchmarking parameters
def my_automatic_layout_parameters(scenario:np.ndarray):
    return {
        "N": scenario.shape[1],
        "M": scenario.shape[2],
        "max_battery_distance": 100,
        "max_battery_time": 100,
        "n_drones": 10,
        "n_ground_stations": 200,
        "n_charging_stations": 50,
    }

In [3]:
# That's very fast to run!
print("starting benchmark")
time_start = time.time()
scenario = load_scenario_npy("Dataset/0001/scenarii/0001_00002.npy")
print("scenario loaded")
device, delta_t, _ = run_benchmark_scenario(scenario, MY_SENSOR_STRATEGY, RandomDroneRoutingStrategy, custom_initialization_parameters = {"burnmap_filename": "./Dataset/0001/burn_map.npy"}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters)
print("Fire detected in ", delta_t, "time steps by device: ", device)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")


starting benchmark
scenario loaded
calling julia optimization model
NEW STRATEGY
N=116
M=287
Set parameter Username
Set parameter LicenseID to value 2630735
Academic license - for non-commercial use only - expires 2026-03-03
Step 1 took 1.4854192091152072 seconds
Step 1 took 0.6992010830435902 seconds
Step 2 took 2.77082904195413 seconds
Step 3 took 0.06828025006689131 seconds
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2724390 rows, 99876 columns and 8039996 nonzeros
Model fingerprint: 0x72af2ae3
Variable types: 33292 continuous, 66584 integer (66584 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [4e-05, 3e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 2642260 rows and 74658 columns
Presolve ti

We can visualize the strategy in action by creating a video of the scenario. 
1. Use the `return_history` parameter or `run_benchmark_scenario` to output the log of ground sensor and drone positions during the benchmark
2. Use the `create_scenario_video` function to compile the video. This can take a couple seconds

In [ ]:
import numpy as np

burnmap = np.load("./Dataset/0001/burn_map.npy", allow_pickle=True)  # Load with pickle enabled

# Extract the first (and only) element from the array, which is a dictionary
burnmap_dict = burnmap.item()  # `.item()` extracts the dictionary from the NumPy array

# Get the actual burn map data stored under the "scenario" key
burnmap_array = burnmap_dict["scenario"]

# Ensure it's stored in a proper NumPy format
np.save("./Dataset/0001/burn_map.npy", burnmap_array.astype(np.float32))  # Save as a clean NumPy array



In [5]:
delta , device , (position_history, ground, charging)  = run_benchmark_scenario(scenario, MY_SENSOR_STRATEGY, RandomDroneRoutingStrategy, custom_initialization_parameters = {"burnmap_filename": "./Dataset/0001/burn_map.npy"}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

calling julia optimization model
NEW STRATEGY
N=116
M=287
Set parameter Username
Set parameter LicenseID to value 2630735
Academic license - for non-commercial use only - expires 2026-03-03
Step 1 took 3.2453584170434624 seconds
Step 1 took 0.5523473748471588 seconds
Step 2 took 3.0837519159540534 seconds
Step 3 took 0.003582332981750369 seconds
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2724390 rows, 99876 columns and 8039996 nonzeros
Model fingerprint: 0x72af2ae3
Variable types: 33292 continuous, 66584 integer (66584 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [4e-05, 3e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 2642260 rows and 74658 columns
Presolve time: 1.56s
Presolved: 82130 rows,

Instead of running a benchmark on a single scenario, we are interested in running the benchmark on all scenarii of a given layout (potentially in parallel!)

1. We use `run_benchmark_scenarii_sequential`

In [ ]:
run_benchmark_scenarii_sequential("Dataset/0001/scenarii/", SensorPlacementOptimization, RandomDroneRoutingStrategy, custom_initialization_parameters_function = get_burnmap_parameters, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=100)

On all of the dataset!

In [10]:
def benchmark_on_sim2real_dataset(dataset_folder_name, ground_placement_strategy, drone_routing_strategy, custom_initialization_parameters_function, custom_step_parameters_function, max_n_scenarii=None, starting_time=0):
    """
    Run benchmarks on a simulation-to-real-world dataset structure.

    Args:
        dataset_folder_name (str): Root folder containing layout folders with scenario data.
        ground_placement_strategy (function): Strategy for placing ground sensors and charging stations.
        drone_routing_strategy (function): Strategy for controlling drone movements.
        ground_parameters (tuple): Parameters for ground placement strategy.
        routing_parameters (tuple): Parameters for routing strategy.
        max_n_scenarii (int, optional): Maximum number of scenarios to process per layout. If None, processes all scenarios.
        starting_time (int, optional): Time step at which the wildfire starts.
    """
    if not dataset_folder_name.endswith('/'):
        dataset_folder_name += '/'
    
    for layout_folder in os.listdir(dataset_folder_name):
        if not os.path.exists(dataset_folder_name + layout_folder + "/scenarii/"):continue
        run_benchmark_scenarii_sequential(dataset_folder_name + layout_folder + "/scenarii/",
                                          ground_placement_strategy, 
                                          drone_routing_strategy, 
                                          custom_initialization_parameters_function, 
                                          custom_step_parameters_function, 
                                          starting_time=starting_time, 
                                          max_n_scenarii=max_n_scenarii)

In [ ]:
benchmark_on_sim2real_dataset("Dataset/", SensorPlacementStrategy, DroneRoutingStrategy, custom_initialization_parameters_function = lambda x: None, custom_step_parameters_function = return_no_custom_parameters, max_n_scenarii=100, starting_time=0)